In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_6b6420fd6a72


pipeline_6b6420fd6a72

In [5]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [6]:
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    sameModel.transform(batchDF).select(getProbability($"probability").alias("negative_probability")).show
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1ac56d3f

+--------------------+
|negative_probability|
+--------------------+
| 0.42844458505393435|
|  0.5144744106785156|
|  0.4926104865820985|
|  0.5094358121915039|
|  0.5144744106785156|
|  0.5282875229714821|
|  0.4377270318681079|
|  0.5267365540365857|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5144744106785156|
|  0.5042018204964029|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
|  0.4934070390133621|
|  0.4377270318681079|
|  0.5471666853609675|
|  0.4086551721636881|
|  0.5094358121915039|
|   0.517056711152886|
|  0.5144744106785156|
|  0.4236810375980825|
|  0.4907085096773568|
|  0.5042018204964029|
|  0.5144744106785156|
|  0.5158569038109135|
+--------------------+

+--------------------+
|negative_probability|
+--------------------+
| 0.48836844052635053|
|  0.5042018204964029|
| 0.46621948928118667|
| 0.47406206630297676|
|  0.5017902887801393|
|  0.5285949589710808|
|  0.4903373054607515|
|  0.5616812248098265|
|  0.5235

In [ ]:
val a = inputDF.withColumn("processingTime",current_timestamp())
    sameModel.transform(a)
    .groupBy($"prediction",window($"processingTime", "10 seconds", "10 seconds")).count().orderBy("window")
    .writeStream
    .outputMode("complete")
    .format("console")
    .option("truncate","false")
    .start().awaitTermination()
    sameModel.transform(inputDF).printSchema()


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+------------------------------------------+-----+
|prediction|window                                    |count|
+----------+------------------------------------------+-----+
|1.0       |[2020-03-10 11:40:10, 2020-03-10 11:40:20]|9    |
|0.0       |[2020-03-10 11:40:10, 2020-03-10 11:40:20]|3    |
+----------+------------------------------------------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+------------------------------------------+-----+
|prediction|window                                    |count|
+----------+------------------------------------------+-----+
|1.0       |[2020-03-10 11:40:10, 2020-03-10 11:40:20]|9    |
|0.0       |[2020-03-10 11:40:10, 2020-03-10 11:40:20]|3    |
|0.0       |[2020-03-10 11:40:20, 2020-03-10 11:40:30]|5    |
|1.0       |[2020-03-10 11:40:20, 2020-03-10 11:40:30]|7    |